## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-16-01-15-38 +0000,startribune,Live: U.S. men’s hockey team beats Germany at ...,https://www.startribune.com/2026-winter-olympi...
1,2026-02-16-01-10-00 +0000,wsj,Asian Currencies Mixed; Could be Supported by ...,https://www.wsj.com/finance/currencies/asian-c...
2,2026-02-16-00-54-00 +0000,wsj,JGB Futures Edge Higher as Investors Assess Ja...,https://www.wsj.com/finance/commodities-future...
3,2026-02-16-00-53-12 +0000,nyt,Diplomatic Feud With China Weighs on Japan’s E...,https://www.nytimes.com/2026/02/15/business/ja...
4,2026-02-16-00-51-00 +0000,wsj,"Oil Consolidates, But Prospects of OPEC+ Suppl...",https://www.wsj.com/finance/commodities-future...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2508/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
102,trump,12
52,ice,7
321,new,7
305,will,6
193,nyc,6


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
12,2026-02-16-00-22-00 +0000,wsj,The U.S. military airlifted on Sunday a miniat...,https://www.wsj.com/politics/national-security...,43
105,2026-02-15-13-39-00 +0000,wsj,While the main focus of the Munich Security Co...,https://www.wsj.com/politics/policy/europeans-...,33
92,2026-02-15-15-32-07 +0000,bbc,'Trump will be gone in three years': Top Democ...,https://www.bbc.com/news/articles/cjrq2r9y278o...,33
46,2026-02-15-21-03-28 +0000,nypost,Multiple additional gloves found by The Post i...,https://nypost.com/2026/02/15/us-news/multiple...,33
158,2026-02-15-02-00-00 +0000,wsj,"Inflation is easing, jobs are holding up, and ...",https://www.wsj.com/economy/federal-reserve-so...,31


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
12,43,2026-02-16-00-22-00 +0000,wsj,The U.S. military airlifted on Sunday a miniat...,https://www.wsj.com/politics/national-security...
46,33,2026-02-15-21-03-28 +0000,nypost,Multiple additional gloves found by The Post i...,https://nypost.com/2026/02/15/us-news/multiple...
105,33,2026-02-15-13-39-00 +0000,wsj,While the main focus of the Munich Security Co...,https://www.wsj.com/politics/policy/europeans-...
158,31,2026-02-15-02-00-00 +0000,wsj,"Inflation is easing, jobs are holding up, and ...",https://www.wsj.com/economy/federal-reserve-so...
111,28,2026-02-15-13-08-14 +0000,nyt,"Gisèle Pelicot, the Woman at the Center of Rap...",https://www.nytimes.com/2026/02/13/magazine/gi...
7,24,2026-02-16-00-33-26 +0000,nypost,Daughters of Irishman held by ICE say father a...,https://nypost.com/2026/02/15/us-news/daughter...
155,24,2026-02-15-02-04-02 +0000,bbc,Russia killed opposition leader Alexei Navalny...,https://www.bbc.com/news/articles/cwyk4lz4e3eo...
39,23,2026-02-15-22-02-44 +0000,nypost,Calif. lawmaker’s Epstein bombshell outs 4 ran...,https://nypost.com/2026/02/15/us-news/californ...
152,22,2026-02-15-03-15-20 +0000,nypost,Roger Goodell thinks his reckless NFL antics w...,https://nypost.com/2026/02/14/business/roger-g...
29,20,2026-02-15-23-01-33 +0000,nypost,Coney Island lands $1M from NYC for revitaliza...,https://nypost.com/2026/02/15/us-news/coney-is...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
